# CPE018 Midterm Exam (1st Sem, A.Y. 2023-2024)


Student Submission Details:
* Name: Danilo Jr. G. Buan
* Section: CPE31S3
* Schedule: Saturday : 10:30 - 1:30
* Instructor:Engr. Roman M. Richard

## Intended Learning Outcomes

By the end of this activity, the student should be able to:
* ILO1: Demonstrate different methods for feature matching and detection learned in class and indepdentently from new sources.
* ILO2: Evaluate the accuracy of different feature matching and detection methods and scrutinize its applicability in solving a given real-life problem.

## Tasks

For this examination, you must create a **mood detection** program with an object-oriented programming approach (same as project CAMEO), it must detect mood changes through the use of algorithms/techniques/schemes learned in class, and from external sources.

In this file, you have to include for each section of your solution your completion of the following:

* Part 1: **Face Detection**: Once your face is detected using any algorithm, it must draw an ROI. The color for the ROI is your choice; however, it must detect for all faces in the frame and draw a corresponding ROI.
* Part 2: **Face Recognition**: The detected face must then be recognized, using any of the provided tools in class, the ROIs must indicate whether it is your face or someone it doesn't recognize.
* Part 3: **Mood Detection**: Use three different feature detection and matching techniques to determine three emotion: happy, sad and neutral. Two of the techniques must be learned from class, and 1 must be one you independently learned.

Properly show through your notebook the output for each part of the exam.

## Procedure and Outputs

Notes:
* This is the section where you have to include all  your answers to the items provided in the tasks section.
* Tasks 1 and 2 contribute directly to ILO1: Demonstrate different methods for feature matching and detection learned in class and indepdentently from new sources.
* Task 3 contributes directly to ILO2: Evaluate the accuracy of different feature matching and detection methods and scrutinize its applicability in solving a given real-life problem.

### Task 1: Face Detection

In [21]:
import cv2

def detect():
    face_cascade_path = r'F:\Desktop\MIDTERMS EXAMS\haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(face_cascade_path)

    camera = cv2.VideoCapture(0)

    while True:
        ret, frame = camera.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            img = cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]

        cv2.imshow("camera", frame)
        if cv2.waitKey(1) & 0xFF == ord(" "):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect()


# Output for Face Detection
![](facedetect.jpg)

### Task 2: Face Recognition

In [ ]:
import numpy as np
import os
import errno
import sys
import cv2

def read_images(path, sz=None):
    c = 0
    X, y = [], []

    for dirname, dirnames, filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                try:
                    if filename == ".directory":
                        continue
                    filepath = os.path.join(subject_path, filename)
                    im = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

                    # Resize the images to the prescribed size if sz is provided
                    if sz is not None:
                        im = cv2.resize(im, sz)

                    X.append(np.asarray(im, dtype=np.uint8))
                    y.append(c)

                except IOError as e:
                    print(f"I/O Error({e.errno}): {e.strerror}")
                except Exception as e:
                    print("Unexpected error:", e)

            c += 1

    return [X, y]

def face_rec():
    names = ['BUAN','UNKNOWN']  # Put your names here for faces to recognize

    # Provide the path to your image dataset folder here
    dataset_path = 'F:\Desktop\MIDTERMS EXAMS\DATASET'

    # Call the function to read images from the dataset directory
    X, y = read_images(dataset_path)
    y = np.asarray(y, dtype=np.int32)

    # Create a face recognition model
    model = cv2.face.EigenFaceRecognizer_create()
    model.train(X, y)

    camera = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier('F:\Desktop\MIDTERMS EXAMS\haarcascade_frontalface_default.xml')

    while True:
        ret, img = camera.read()
        if not ret:
            break

        faces = face_cascade.detectMultiScale(img, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
            roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

            try:
                params = model.predict(roi)
                label = names[params[0]]
                cv2.putText(img, label, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            except:
                cv2.putText(img, "UNKNOWN", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow("camera", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()

# Output for Face Recognition
![](facerecog.jpg)

### Task 3: Mood Detection

In [23]:
import cv2
import numpy as np
from keras.models import model_from_json

emotion_dict = {0: "Happy", 1: "Neutral", 2: "Sad"}


json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)


emotion_model.load_weights("model/emotion_model.h5")
print("Loaded model from disk")

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)


    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)


        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))


        if maxindex in emotion_dict:
            cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Output For Mood Detection
## Happy
![](mood_happy.jpg)
## Neutral
![](mood_neutral.jpg)
## Sad
![](mood_sad.jpg)
![](try_mood.jpg)
![](try2_mood.jpg)


## Analysis

For the three different techniques you used in face detection, provide an in-depth analysis. 

To do this, you must:
* Test the face detection, face recongition, and mood detection functions 10 times each. Only the mood detection will have components for 10 tests for each different technique used.
* Create a table containing the 10 tests (like shown below) for each task.
* Analyze each output by identifying the accuracy and providing your observations.

In [1]:
# Import the module for tabulating the data
from tabulate import tabulate
 
# Create a list for content of the table
test_results = [
    ["1", "Happy", "Happy", 1],
    ["2", "Happy", "Happy", 1],
    ["3", "Happy", "Happy", 1],
    ["4", "Neutral", "Neutral", 1],
    ["5", "Neutral", "Neutral", 1],
    ["6", "Neutral", "Neutral", 1],
    ["7", "Neutral", "Neutral", 1],
    ["8", "Sad", "Sad", 1],
    ["9", "Sad", "Sad", 1],
    ["10", "Sad", "Neutral", 0]
]
 
# Create a list for the headers of your table
header = ["Test #", "Expected", "Actual", "Score"]
 
# display table
print("Task 3A: Mood Detection using XYZ Algorithm")
print(tabulate(test_results, headers=header, tablefmt="grid"))

# Calculate for the accuracy
total = 0
for i in test_results:
    total += i[3]
print("Accuracy: ", round(total/len(test_results)*100,2))

Task 3A: Mood Detection using XYZ Algorithm
+----------+------------+----------+---------+
|   Test # | Expected   | Actual   |   Score |
+==========+============+==========+=========+
|        1 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        2 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        3 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        4 | Neutral    | Neutral  |       1 |
+----------+------------+----------+---------+
|        5 | Neutral    | Neutral  |       1 |
+----------+------------+----------+---------+
|        6 | Neutral    | Neutral  |       1 |
+----------+------------+----------+---------+
|        7 | Neutral    | Neutral  |       1 |
+----------+------------+----------+---------+
|        8 | Sad        | Sad      |       1 |
+----------+------------+----------+---------+
|        9 | Sad        | Sad      |       1 |
+----------+----

## Summary and Lessons Learned

         For this midterm exam, my skills were put to the test in creating a mood detection program. At first, my knowledge on face detection and recognition were jogged up and refreshed that helped me in the last task that was to implement a mood detection program. Since I myself is not very good at coding, watching youtube videos and researching about mood detection really helped me implement this successfully. After performing this activity, this helped me refreshen my knowledge about face detection and face recognition and also the addition of mood detection. Mood detection is somewhat similar to face recognition since training models were involved. The dataset that I have used for the three moods (happy, neutral, and sad) were available on the internet so it was a little bit relieving. The use of other modules such as keras that is used for deep learning really helped a lot for successfully implementing the last task. As seen on the table above, the accuracy is 90% which is very high. Also, I tried to use pictures of other people to test if it can detect their moods also.

---

**Proprietary Clause**

Property of the Technological Institute of the Philippines (T.I.P.). No part of the materials made and uploaded in this learning management system by T.I.P. may be copied, photographed, printed, reproduced, shared, transmitted, translated or reduced to any electronic medium or machine-readable form, in whole or in part, without prior consent of T.I.P.

Prepared by Engr. RMR